# TRAINING 
---
## IMPORT 

In [ ]:
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard

import os

## Load Data

In [ ]:
with open('../data/train_data.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [ ]:
df.head()

,Id_x,OwnerUserId_x,CreationDate_x,Score_x,Body_x,Id_y,OwnerUserId_y,CreationDate_y,Score_y,Title,Body_y
0,497,50.0,2008-08-02T16:56:53Z,4,<p>open up a terminal (Applications-&gt;Utilit...,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,518,153.0,2008-08-02T17:42:28Z,2,<p>I haven't been able to find anything that d...,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
2,3040,457.0,2008-08-06T03:01:23Z,12,<p>Unfortunately the only API that isn't depre...,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
3,195170,745.0,2008-10-12T07:02:40Z,1,<p>There must be a method in Cocoa to get a li...,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
4,536,161.0,2008-08-02T18:49:07Z,9,<p>You can use ImageMagick's convert utility f...,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...


## Split train/ test

In [ ]:
# Target = pertinence donnée à la réponse
X_train, X_test, y_train, y_test = train_test_split(df[['Id_x', 'Title', 'Body_y', 'Body_x']], df['Score_x'], test_size=0.2, random_state=0)

In [ ]:
# Mean length of body x and y
average_x = sum(len(word) for word in X_train['Body_x']) / len(X_train['Body_x'])
average_y = sum(len(word) for word in X_train['Body_y']) / len(X_train['Body_y'])
average_title = sum(len(word) for word in X_train['Title']) / len(X_train['Title'])
print(average_x)
print(average_y)
print(average_title)

774.0200697229444
1212.5857537764484
51.093487755430246


## Preprocessing

In [ ]:
def vectorize_data(X_train, X_test, max_features=1000):    
    vectorizer = CountVectorizer(max_features=max_features)

    X_train_vect = X_train
    X_test_vect = X_test

    X_train_vect['Title'] = vectorizer.fit_transform(X_train_vect['Title'])
    X_test_vect['Title'] = vectorizer.transform(X_test_vect['Title'])

    X_train_vect['Body_x'] = vectorizer.fit_transform(X_train_vect['Body_x'])
    X_test_vect['Body_x'] = vectorizer.transform(X_test_vect['Body_x'])

    X_train_vect['Body_y'] = vectorizer.fit_transform(X_train_vect['Body_y'])
    X_test_vect['Body_y'] = vectorizer.transform(X_test_vect['Body_y'])

    return X_train_vect, X_test_vect

In [ ]:
X_train_vect, X_test_vect = vectorize_data(X_train, X_test, max_features=1000)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
# Même longeur de texte
# maxlenBodyX = 750
# maxlenBodyY = 750
# maxLenTitle = 60

# x_train_pad = X_train_vect
# x_test_pad = X_test_vect

# x_train_pad['Title'] = sequence.pad_sequences(x_train_pad['Title'], maxLenTitle)
# x_train_pad['Body_x'] = sequence.pad_sequences(x_train_pad['Body_x'], maxlenBodyX)
# x_train_pad['Body_y'] = sequence.pad_sequences(x_train_pad['Body_y'], maxlenBodyY)

# x_test_pad['Title'] = sequence.pad_sequences(x_test_pad['Title'], maxLenTitle)
# x_test_pad['Body_x'] = sequence.pad_sequences(x_test_pad['Body_x'], maxlenBodyX)
# x_test_pad['Body_y'] = sequence.pad_sequences(x_test_pad['Body_y'], maxlenBodyY)


## Create model

In [ ]:
def build_model(input_shape, DROPOUT_RATE = 0.2, UNITS = 64, NUM_CLASSES = 2):

    model = keras.Sequential()
    model.add(Dense(units=50, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.build(input_shape)
    model.summary()
    return model

In [ ]:
X_train_vect[0].shape[1]

4

In [ ]:
model = build_model(X_train_vect.shape)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (4, 50)                   250       
_________________________________________________________________
dense_7 (Dense)              (4, 1)                    51        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


## Compilation

In [ ]:
INIT_LR = 0.001
EPOCHS = 10

In [ ]:
optimizer = Adam(learning_rate=INIT_LR, decay=INIT_LR/EPOCHS)

In [ ]:
model.compile(
  optimizer = optimizer,
  loss = "binary_crossentropy",
  metrics = ["accuracy"]
)

## Entrainement

In [ ]:
def train_model(X_train, y_train, EPOCHS = 100, BATCH_SIZE = 128, patience=5):
    # For tensorboard
    os.makedirs("./logs",exist_ok=True)
    tensorboard = TensorBoard(log_dir=os.path.join('./logs'), histogram_freq=0,
                                write_graph=True, write_images=False,profile_batch = 100000000)

    callbacks = [tensorboard]

    history = model.fit(X_train, y_train, callbacks=callbacks)
    return history

In [ ]:
X_train_vect

<4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [ ]:
history = train_model(X_train_vect, y_train, EPOCHS = 100, BATCH_SIZE = 128, patience=5)

ValueError: Data cardinality is ambiguous:
  x sizes: 4
  y sizes: 789697
Please provide data which shares the same first dimension.

## Analyse des résultats

In [ ]:
y_test_pred = model.predict_classes(X_test_vect)

In [ ]:
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_test, y_test_pred)
print(cf_matrix)

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e80043e2-6875-4b65-a196-a0ffb97a1282' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>